In [ ]:
# script to parse html-data from ziengs.nl and insert into sqlite ziengs.db

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bs4 import BeautifulSoup
import sqlite3
import json
import re

In [69]:
con = sqlite3.connect('ziengs.db')

with con:
    
    cur = con.cursor()    
                 
    with open('crawl_ziengs.nl_2016-05-30T23-15-20.jl') as file:
        
        product_pages = []
        product_listings = []
        
        for nr, line in enumerate(file):

            data = json.loads(line)
            soup = BeautifulSoup(data['body'],"lxml")

            #two different page_types to parse: this is product_detail page. This data goes to separate tables in ziengs.db
            if data['page_type'] == 'product_detail':           

                product_id = soup.find("input",{"id":"hdnProductId"}).get("value")
                title = soup.find("h1", {"itemprop":"name"}).contents[0]
                category = soup.find("meta",{"itemprop":"category"}).get("content")
                brand = soup.find("meta", {"itemprop":"brand"}).get("content")
                price = soup.find("meta", {"itemprop":"price"}).get("content")
                page_url = data['page_url']
                href_re = re.match(r'http://www.ziengs.nl/(.*)',page_url).group(1)
                image = soup.find("meta", {"property":"og:image"}).get("content")
                page_type = data['page_type']
                crawled_at = data['crawled_at']
                try: 
                    details = soup.find("div", {"class":"usp"}).contents[0]
                    if not isinstance(details,str): 
                         details = details[0]
                except: details = ' '

                product_pages.append((product_id,title,category,brand,price,page_url,href_re,image,page_type,crawled_at,details))

            #two different page_types to parse: this is product_listing page. This data goes to separate tables in ziengs.db    
            if data['page_type'] == 'product_listing':        

                nr_products = soup.find("div",{"class":"productCount"}).find("span").contents[0].strip()

                items = soup.findAll("div",{"class":["item","item last","item vanvoor","item vanvoor last"]})
                
                crawled_at = data['crawled_at']
                product_category = ", ".join(data['product_category'])
                ordering = data['ordering']
                page_number = data['page_number']
                page_type = data['page_type']
                page_url = data['page_url']
                
                position = 0 + (int(page_number)-1) * 24
                for p_nr, item in enumerate(items):
                    product_id = item.get("rel")
                    color_id = item.get("rev")
                    if product_id is not None:
                        href = item.find("ul",{"data-colorid":color_id})
                        href2 = href.find("a").get("href")
                        try: 
                            href_re = re.match(r'.*/([A-Za-z0-9].*)',href2).group(1)
                        except: #code om te testen
                            print item.prettify()
                            print nr, p_nr, href, product_id
                            print page_url
                            print href2
                            assert False
                            
                        position += 1
                        product_listings.append((page_url, nr_products, page_number, position, ordering, product_id, href_re, product_category, page_type, crawled_at))
                        
        cur.executemany('insert into product_pages4 values (?,?,?,?,?,?,?,?,?,?,?)', product_pages)
        cur.executemany('insert into product_listings4 values (?,?,?,?,?,?,?,?,?,?)', product_listings)
        con.commit()

        file.close()
    
con.close()

nr: 0
pdp_pages: 0
p_listings: 1
other: 0
nr: 100
pdp_pages: 78
p_listings: 23
other: 0
nr: 200
pdp_pages: 171
p_listings: 30
other: 0
nr: 300
pdp_pages: 266
p_listings: 35
other: 0
nr: 400
pdp_pages: 362
p_listings: 39
other: 0
nr: 500
pdp_pages: 458
p_listings: 43
other: 0
nr: 600
pdp_pages: 555
p_listings: 46
other: 0
nr: 700
pdp_pages: 649
p_listings: 52
other: 0
nr: 800
pdp_pages: 745
p_listings: 56
other: 0
nr: 900
pdp_pages: 839
p_listings: 62
other: 0
nr: 1000
pdp_pages: 935
p_listings: 66
other: 0
nr: 1100
pdp_pages: 1032
p_listings: 69
other: 0
nr: 1200
pdp_pages: 1128
p_listings: 73
other: 0
nr: 1300
pdp_pages: 1225
p_listings: 76
other: 0
nr: 1400
pdp_pages: 1318
p_listings: 83
other: 0
nr: 1500
pdp_pages: 1407
p_listings: 94
other: 0
nr: 1600
pdp_pages: 1493
p_listings: 108
other: 0
nr: 1700
pdp_pages: 1590
p_listings: 111
other: 0
nr: 1800
pdp_pages: 1682
p_listings: 119
other: 0
nr: 1900
pdp_pages: 1772
p_listings: 129
other: 0
nr: 2000
pdp_pages: 1858
p_listings: 143
ot